In [1]:
from nuscenes import NuScenes

#Load ground truth data
nusc = NuScenes(version='v1.0-trainval', dataroot='/home/albhaisi/Desktop/WS/data', verbose=True)

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 25.664 seconds.
Reverse indexing ...
Done reverse indexing in 5.9 seconds.


In [2]:
#Load_results
import json
from nuscenes.eval.common.data_classes import EvalBoxes
from nuscenes.eval.tracking.data_classes import TrackingBox

#give tracker path
result_path = '/home/albhaisi/Desktop/senarios/final_results/4_turned_car/Voxel01.json'
instance_to_find= '52504c169efd4a0bb1437bfb8ed5bdb9'

def load_predictions(result_path):
        #Choose which scene results to upload 
        with open(result_path) as f:
                data = json.load(f)
                result = data['results']
        all_results = EvalBoxes.deserialize(data['results'], TrackingBox)
        print(all_results)
        meta = data['meta'] 
        # How much sample in this result and number of boxes
        print("Loaded results from {}. Found detections for {} samples."
              .format(result_path, len(all_results.sample_tokens)))

        return result

prediction_results= load_predictions(result_path)

#Check_results
print(prediction_results)

EvalBoxes with 708 boxes across 40 samples
Loaded results from /home/albhaisi/Desktop/senarios/final_results/4_turned_car/Voxel01.json. Found detections for 40 samples.
{'418cbb70159341c1ada4f3ba1fb69713': [{'translation': [363.4235710607566, 1086.4217941021245, 0.7193708562917144], 'size': [1.8896936178207397, 4.413407802581787, 1.665687918663025], 'velocity': [0.0, 0.0], 'rotation': [0.8330171913854183, 0.0, 0.0, 0.5532471046976654], 'tracking_score': 0.8306690454483032, 'sample_token': '418cbb70159341c1ada4f3ba1fb69713', 'tracking_id': 'car_43_0', 'tracking_name': 'car'}, {'translation': [362.73952536605367, 1091.95066380746, 0.6752579658252477], 'size': [1.8408061265945435, 4.4202728271484375, 1.5678820610046387], 'velocity': [0.0, 0.0], 'rotation': [0.8227798400212609, 0.0, 0.0, 0.5683602157563357], 'tracking_score': 0.8293432593345642, 'sample_token': '418cbb70159341c1ada4f3ba1fb69713', 'tracking_id': 'car_43_1', 'tracking_name': 'car'}, {'translation': [368.41565861451767, 1114.

In [3]:
from collections import defaultdict
from typing import List, Dict, Tuple

import numpy as np

from nuscenes.eval.common.data_classes import MetricData, EvalBox
from nuscenes.eval.common.utils import center_distance, scale_iou
from nuscenes.eval.detection.constants import DETECTION_NAMES, ATTRIBUTE_NAMES, TP_METRICS

#Edit Detectionbox

class DetectionBox(EvalBox):
    """ Data class used during detection evaluation. Can be a prediction or ground truth."""

    def __init__(self,
                 sample_token: str = "",                 
                 sample_annotation_token: str = "", # sample_annotation_token only for gt boxes.
                 translation: Tuple[float, float, float] = (0, 0, 0),
                 size: Tuple[float, float, float] = (0, 0, 0),
                 rotation: Tuple[float, float, float, float] = (0, 0, 0, 0),
                 velocity: Tuple[float, float] = (0, 0),
                 ego_translation: [float, float, float] = (0, 0, 0),  # Translation to ego vehicle in meters.
                 num_pts: int = -1,  # Nbr. LIDAR or RADAR inside the box. Only for gt boxes.
                 tracking_name: str = 'car',  # The class name used in the detection challenge.
                 detection_name: str = 'car',
                 detection_score: float = -1.0,
                 tracking_score: float = -1.0):  # GT samples do not have a score.
                 
          

        super().__init__(sample_token, translation, size, rotation, velocity, ego_translation, num_pts)

        assert tracking_name is not None, 'Error: detection_name cannot be empty!'
        assert tracking_name in DETECTION_NAMES, 'Error: Unknown detection_name %s' % tracking_name


        assert type(tracking_score) == float, 'Error: detection_score must be a float!'
        assert not np.any(np.isnan(tracking_score)), 'Error: detection_score may not be NaN!'

        # Assign.
        self.tracking_name = tracking_name
        self.detection_name= detection_name
        self.tracking_score = tracking_score
        self.detection_score = detection_score
    def __eq__(self, other):
        return (self.sample_token == other.sample_token and
                self.sample_annotation_token == other.sample_annotation_token and
                self.translation == other.translation and
                self.size == other.size and
                self.rotation == other.rotation and
                self.velocity == other.velocity and
                self.ego_translation == other.ego_translation and
                self.num_pts == other.num_pts and
                self.tracking_name == other.tracking_name and
                self.detection_name == other.detection_name and
                self.detection_score == other.detection_score and
                self.tracking_score == other.tracking_score)

    def serialize(self) -> dict:
        """ Serialize instance into json-friendly format. """
        return {
            'sample_token': self.sample_token,
            'sample_annotation_token': self.sample_annotation_token,
            'translation': self.translation,
            'size': self.size,
            'rotation': self.rotation,
            'velocity': self.velocity,
            'ego_translation': self.ego_translation,
            'ego_dist': self.ego_dist, # return ego distance as well
            'num_pts': self.num_pts,
            'tracking_name': self.tracking_name,
            'detection_name': self.detection_name,
            'tracking_score': self.tracking_score,
            'detection_score': self.detection_score
    
        }

    @classmethod
    def deserialize(cls, content: dict):
        """ Initialize from serialized content. """
        return cls(sample_token=content['sample_token'],
                   sample_annotation_token= "" if 'sample_annotation_token' not in content
                   else content['sample_annotation_token'],
                   translation=tuple(content['translation']),
                   size=tuple(content['size']),
                   rotation=tuple(content['rotation']),
                   velocity=tuple(content['velocity']),
                   ego_translation=(0.0, 0.0, 0.0) if 'ego_translation' not in content
                   else tuple(content['ego_translation']),
                   num_pts=-1 if 'num_pts' not in content else int(content['num_pts']),
                   tracking_name=content['tracking_name'],
                   detection_name=content['tracking_name'],
                   detection_score=-1.0 if 'detection_score' not in content else float(content['tracking_score']),
                   tracking_score=-1.0 if 'tracking_score' not in content else float(content['tracking_score']))
    
class TrackingBox(EvalBox):
    """ Data class used during tracking evaluation. Can be a prediction or ground truth."""

    def __init__(self,
                 sample_token: str = "",
                 translation: Tuple[float, float, float] = (0, 0, 0),
                 size: Tuple[float, float, float] = (0, 0, 0),
                 rotation: Tuple[float, float, float, float] = (0, 0, 0, 0),
                 velocity: Tuple[float, float] = (0, 0),
                 ego_translation: Tuple[float, float, float] = (0, 0, 0),  # Translation to ego vehicle in meters.
                 num_pts: int = -1,  # Nbr. LIDAR or RADAR inside the box. Only for gt boxes.
                 tracking_id: str = instance_to_find,  # Instance id of this object.
                 tracking_name: str = '',  # The class name used in the tracking challenge.
                 tracking_score: float = -1.0):  # Does not apply to GT.

        super().__init__(sample_token, translation, size, rotation, velocity, ego_translation, num_pts)

        #assert tracking_name is not None, 'Error: tracking_name cannot be empty!'
        #assert tracking_name in TRACKING_NAMES, 'Error: Unknown tracking_name %s' % tracking_name

        assert type(tracking_score) == float, 'Error: tracking_score must be a float!'
        assert not np.any(np.isnan(tracking_score)), 'Error: tracking_score may not be NaN!'

        # Assign.
        self.tracking_id = tracking_id
        self.tracking_name = tracking_name
        self.tracking_score = tracking_score

    def __eq__(self, other):
        return (self.sample_token == other.sample_token and
                self.translation == other.translation and
                self.size == other.size and
                self.rotation == other.rotation and
                self.velocity == other.velocity and
                self.ego_translation == other.ego_translation and
                self.num_pts == other.num_pts and
                self.tracking_id == other.tracking_id and
                self.tracking_name == other.tracking_name and
                self.tracking_score == other.tracking_score)

    def serialize(self) -> dict:
        """ Serialize instance into json-friendly format. """
        return {
            'sample_token': self.sample_token,
            'translation': self.translation,
            'size': self.size,
            'rotation': self.rotation,
            'velocity': self.velocity,
            'ego_translation': self.ego_translation,
            'num_pts': self.num_pts,
            'tracking_id': self.tracking_id,
            'tracking_name': self.tracking_name,
            'tracking_score': self.tracking_score
        }

    @classmethod
    def deserialize(cls, content: dict):
        """ Initialize from serialized content. """
        return cls(sample_token=content['sample_token'],
                   translation=tuple(content['translation']),
                   size=tuple(content['size']),
                   rotation=tuple(content['rotation']),
                   velocity=tuple(content['velocity']),
                   ego_translation=(0.0, 0.0, 0.0) if 'ego_translation' not in content
                   else tuple(content['ego_translation']),
                   num_pts=-1 if 'num_pts' not in content else int(content['num_pts']),
                   tracking_id=content['tracking_id'],
                   tracking_name=content['tracking_name'],
                   tracking_score=-1.0 if 'tracking_score' not in content else float(content['tracking_score']))

In [4]:

import tqdm



def get_all_samples_in_scene():
        all_samples_in_scene =[]
        #To know number of annotation for all instances
        for samples in prediction_results:
                all_samples_in_scene.append(samples)
        return all_samples_in_scene

print(get_all_samples_in_scene())

#To get timestamp , sensors,annotations and other information about samples
def get_all_sample_information_in_scene():
        sample_alist=[]
        
        for sample_token in tqdm.tqdm(get_all_samples_in_scene(), leave=False):
                sample = nusc.get('sample', sample_token)
                sample_alist.append(sample)
        return sample_alist


def get_all_annotations():
        sample_annotation_tokens_list=[]
        for annotation_tokens in get_all_sample_information_in_scene():
                sample_annotation_tokens1 = annotation_tokens['anns']  # Get all annotations for all instances
                sample_annotation_tokens_list.append(sample_annotation_tokens1)
        return sample_annotation_tokens_list

def get_all_information_for_all_instances():
        samples_annotation_alist=[]
        for sampe_annotations_token in get_all_annotations():
                for s in sampe_annotations_token:
                        sample_annotation = nusc.get('sample_annotation', s)
                        samples_annotation_alist.append(sample_annotation)
                        #print(sample_annotation)
        return samples_annotation_alist

def find_one_instance(find_instance):
    
    sample = nusc.get('instance', find_instance)
    numb_ann = sample['nbr_annotations']
    
    print("Number of annoation for this instance Found is {} annotations."
        .format(numb_ann))
    return sample

#Give the instance id
my_insatnce = find_one_instance(instance_to_find)
print(my_insatnce)


['418cbb70159341c1ada4f3ba1fb69713', '1a89f57231644ef9a941a51bc4322bdd', '25a777903d554457b6e4e5e437f21125', '92bf51f4b2c447b5a8e068e25cf1f980', '66acf00fb96648bd8d7493be238ecc8d', 'a893b19c605e41f8b9f625cd5d968960', '96d621f9ae4d44f0bf45688877a2d8fc', '9c1fc0d8c93b4a8584cc4f41d2863667', '8010cad5488d4f449a45b8ba24f97fb3', 'b416110f0fcc4aeaad2c3ea13ff0f519', '88918bab1cb841f3a0fbb92a1110fc47', 'eb9d7bc242d14e92a46c1037126cb204', '1b9f1ac3f22b410ab8d5252ebddd78d7', 'e79bf81d98084e178032acbcdd9cb909', '780457151a8a4d3a83b09dd1c7e9403b', '60321e7996b64926bfb9afd9c53b4170', '0799844f6afa4ac1840181d0c8b26f08', '6489d3f46b87471086ba9c5c1ef9266b', 'abc3bcd0053842e482d2296fc55cc405', '76919271ab6d402b908a31648191a5a4', 'a1914259fc2542dbaf82def95b83f9d5', '586d6516ca7e4c528a6a59eb470e17a8', '113f0ef12c2d455ebd676580a8bb9d57', 'f5fa686f58434bb3b199e4d87be19261', '4e39dbe32cc443a4bfd5fb620628472c', '18e40c605a454373ad2074ec5429745c', '2ee49140790544d8b01f52813ba9444c', '1fde5dec10c4471d97b84c1074

In [5]:
#Load_ground_truth_for_one_scene
#choose instance



from typing import Dict, Tuple
import numpy as np
import tqdm
from pyquaternion import Quaternion
from nuscenes.eval.detection.utils import category_to_detection_name
from nuscenes.utils.data_classes import Box
from nuscenes.utils.geometry_utils import points_in_box
from nuscenes.utils.splits import create_splits_scenes
from nuscenes.eval.common.data_classes import EvalBoxes
from nuscenes.eval.tracking.data_classes import TrackingBox


eval_set: str
verbose: bool = True

eval_set='val'

def get_all_samples_in_scene():
        all_samples_in_scene =[]
        #To know number of annotation for all instances
        for samples in prediction_results:
                all_samples_in_scene.append(samples)
        return all_samples_in_scene

trans_alist=[]
lidar_pts_list=[]
visibility_alist=[]
size_gt_alist = []
rotation_gt_alist=[]

def load_gt(nusc: NuScenes, eval_split: str, box_cls, verbose: bool = False) -> EvalBoxes:
    """
    Loads ground truth boxes from DB.
    :param nusc: A NuScenes instance.
    :param eval_split: The evaluation split for which we load GT boxes.
    :param box_cls: Type of box to load, e.g. DetectionBox or TrackingBox.
    :param verbose: Whether to print messages to stdout.
    :return: The GT boxes.
    """
    # Init.
    if box_cls == DetectionBox:
        attribute_map = {a['token']: a['name'] for a in nusc.attribute}

    if verbose:
        print('Loading annotations for {} split from nuScenes version: {}'.format(eval_split, nusc.version))
    # Read out all sample_tokens in DB.
    sample_tokens_all = get_all_samples_in_scene()
    assert len(sample_tokens_all) > 0, "Error: Database has no samples!"

    # Only keep samples from this split.
    splits = create_splits_scenes()

    # Check compatibility of split with nusc_version.
    version = nusc.version
    if eval_split in {'train', 'val', 'train_detect', 'train_track'}:
        assert version.endswith('trainval'), \
            'Error: Requested split {} which is not compatible with NuScenes version {}'.format(eval_split, version)
    elif eval_split in {'mini_train', 'mini_val'}:
        assert version.endswith('mini'), \
            'Error: Requested split {} which is not compatible with NuScenes version {}'.format(eval_split, version)
    elif eval_split == 'test':
        assert version.endswith('test'), \
            'Error: Requested split {} which is not compatible with NuScenes version {}'.format(eval_split, version)
    else:
        raise ValueError('Error: Requested split {} which this function cannot map to the correct NuScenes version.'
                         .format(eval_split))

    if eval_split == 'test':
        # Check that you aren't trying to cheat :).
        assert len(nusc.sample_annotation) > 0, \
            'Error: You are trying to evaluate on the test set but you do not have the annotations!'

    sample_tokens = []
    for sample_token in sample_tokens_all:
        scene_token = nusc.get('sample', sample_token)['scene_token']
        scene_record = nusc.get('scene', scene_token)
        if scene_record['name'] in splits[eval_split]:
            sample_tokens.append(sample_token)

    all_annotations = EvalBoxes()

    # Load annotations and filter predictions and annotations.
    tracking_id_set = set()
    for sample_token in tqdm.tqdm(sample_tokens, leave=verbose):

        sample = nusc.get('sample', sample_token)
        sample_annotation_tokens = sample['anns']
        

        sample_boxes = []
        for sample_annotation_token in sample_annotation_tokens:

            sample_annotation = nusc.get('sample_annotation', sample_annotation_token)       #get_all_information
            #if instance_to_find == sample_annotation['instance_token']:
            #        print(sample_annotation['instance_token'])
            #print(sample_annotation)
            if box_cls == DetectionBox:
                # Get label name in detection task and filter unused labels.
                tracking_name = category_to_detection_name(sample_annotation['category_name'])   
                if tracking_name is None:
                    continue

                
                if instance_to_find == sample_annotation['instance_token']:
                    sample_boxes.append(
                        box_cls(
                            
                            sample_token=sample_token,
                            translation=sample_annotation['translation'],
                            size=sample_annotation['size'],
                            rotation=sample_annotation['rotation'],
                            velocity=nusc.box_velocity(sample_annotation['token'])[:2],
                            num_pts=sample_annotation['num_lidar_pts'],
                            tracking_name=tracking_name,
                            tracking_score=-1.0,  # GT samples do not have a score.
                        )
                    )
                    trans_gt = sample_annotation['translation']
                    trans_alist.append(trans_gt)
                    lidar_pts= sample_annotation['num_lidar_pts']
                    lidar_pts_list.append(lidar_pts)
                    visibility = sample_annotation['visibility_token']
                    visibility_alist.append(visibility)
                    size_gt = sample_annotation['size']
                    size_gt_alist.append(size_gt)
                    rotation_gt= sample_annotation['rotation']
                    rotation_gt_alist.append(rotation_gt)
            elif box_cls == TrackingBox:
                # Use nuScenes token as tracking id.
                tracking_id = sample_annotation['instance_token']
                
                tracking_id_set.add(tracking_id)

                # Get label name in detection task and filter unused labels.
                # Import locally to avoid errors when motmetrics package is not installed.
                from nuscenes.eval.tracking.utils import category_to_tracking_name
                tracking_name = category_to_tracking_name(sample_annotation['category_name'])
                if tracking_name is None:
                    continue
                if instance_to_find == sample_annotation['instance_token']:
                    sample_boxes.append(
                        box_cls(
                            sample_token=sample_token,
                            translation=sample_annotation['translation'],
                            size=sample_annotation['size'],
                            rotation=sample_annotation['rotation'],
                            velocity=nusc.box_velocity(sample_annotation['token'])[:2],
                            num_pts=sample_annotation['num_lidar_pts'],
                            tracking_id=tracking_id,
                            tracking_name=tracking_name,
                            tracking_score=-1.0  # GT samples do not have a score.
                        )
                    )
                    print(sample_boxes)
            else:
                raise NotImplementedError('Error: Invalid box_cls %s!' % box_cls)

        all_annotations.add_boxes(sample_token, sample_boxes)

    if verbose:
        print("Loaded ground truth annotations for {} samples.".format(len(all_annotations.sample_tokens)))

    return all_annotations

gt_boxes = load_gt(nusc, eval_set, TrackingBox, verbose=verbose)

print(gt_boxes)   



Loading annotations for val split from nuScenes version: v1.0-trainval


100%|██████████| 40/40 [00:00<00:00, 175.90it/s]

[{'sample_token': '418cbb70159341c1ada4f3ba1fb69713', 'translation': [352.627, 1067.133, 0.804], 'size': [2.014, 4.717, 2.007], 'rotation': [0.7874678089713519, 0.0, 0.0, 0.6163557818613032], 'velocity': array([2.52683119, 8.83501188]), 'ego_translation': (0, 0, 0), 'num_pts': 26, 'tracking_id': '52504c169efd4a0bb1437bfb8ed5bdb9', 'tracking_name': 'car', 'tracking_score': -1.0}]
[{'sample_token': '1a89f57231644ef9a941a51bc4322bdd', 'translation': [353.763, 1071.105, 0.787], 'size': [2.014, 4.717, 2.007], 'rotation': [0.7874678089713519, 0.0, 0.0, 0.6163557818613032], 'velocity': array([2.39407827, 8.36716136]), 'ego_translation': (0, 0, 0), 'num_pts': 27, 'tracking_id': '52504c169efd4a0bb1437bfb8ed5bdb9', 'tracking_name': 'car', 'tracking_score': -1.0}]
[{'sample_token': '25a777903d554457b6e4e5e437f21125', 'translation': [354.9, 1075.077, 0.769], 'size': [2.014, 4.717, 2.007], 'rotation': [0.7874678089713519, 0.0, 0.0, 0.6163557818613032], 'velocity': array([2.27487444, 7.95155188]), '

In [6]:
#Load_results
import json

from nuscenes.eval.tracking.data_classes import TrackingConfig
from nuscenes.eval.detection.data_classes import DetectionConfig

#Upload tracking config
with open('/home/albhaisi/Desktop/Presentation/for_automatic_test/results/tracking_nips_2023.json', 'r') as _f:
    cfg = TrackingConfig.deserialize(json.load(_f))

#Upload detection config
with open('/home/albhaisi/Desktop/Presentation/for_automatic_test/base_codes/Final_codes/detection_cvpr_2023.json', 'r') as _f:
    cfg2 = DetectionConfig.deserialize(json.load(_f))

def load_prediction(result_path: str, max_boxes_per_sample: int, box_cls, verbose: bool = False) \
        -> Tuple[EvalBoxes, Dict]:
    """
    Loads object predictions from file.
    :param result_path: Path to the .json result file provided by the user.
    :param max_boxes_per_sample: Maximim number of boxes allowed per sample.
    :param box_cls: Type of box to load, e.g. DetectionBox or TrackingBox.
    :param verbose: Whether to print messages to stdout.
    :return: The deserialized results and meta data.
    """

    # Load from file and check that the format is correct.
    with open(result_path) as f:
        data = json.load(f)
    assert 'results' in data, 'Error: No field `results` in result file. Please note that the result format changed.' \
                              'See https://www.nuscenes.org/object-detection for more information.'

    # Deserialize results and get meta data.
    all_results = EvalBoxes.deserialize(data['results'], box_cls)
    meta = data['meta']
    if verbose:
        print("Loaded results from {}. Found detections for {} samples."
              .format(result_path, len(all_results.sample_tokens)))

    # Check that each sample has no more than x predicted boxes.
    for sample_token in all_results.sample_tokens:
        assert len(all_results.boxes[sample_token]) <= max_boxes_per_sample, \
            "Error: Only <= %d boxes per sample allowed!" % max_boxes_per_sample

    return all_results, meta


pred_boxes, meta = load_prediction(result_path, cfg.max_boxes_per_sample, TrackingBox,
                                                verbose=verbose)
print(load_prediction(result_path,cfg.max_boxes_per_sample,TrackingBox))

pred_boxes2, meta2 = load_prediction(result_path, cfg.max_boxes_per_sample, DetectionBox,
                                                     verbose=verbose)
gt_boxes2 = load_gt(nusc, eval_set, DetectionBox, verbose=verbose)

assert set(pred_boxes2.sample_tokens) == set(gt_boxes2.sample_tokens), \
            "Samples in split don't match samples in predicted tracks."


Loaded results from /home/albhaisi/Desktop/senarios/final_results/4_turned_car/Voxel01.json. Found detections for 40 samples.
(EvalBoxes with 708 boxes across 40 samples, {'use_camera': False, 'use_lidar': True, 'use_radar': False, 'use_map': False, 'use_external': False})
Loaded results from /home/albhaisi/Desktop/senarios/final_results/4_turned_car/Voxel01.json. Found detections for 40 samples.
Loading annotations for val split from nuScenes version: v1.0-trainval


100%|██████████| 40/40 [00:00<00:00, 6961.79it/s]

Loaded ground truth annotations for 40 samples.


In [7]:
from nuscenes.eval.common.loaders import add_center_dist,filter_eval_boxes
from nuscenes.eval.tracking.loaders import create_tracks 
from nuscenes.eval.detection.data_classes import DetectionMetricDataList
from nuscenes.eval.detection.algo import accumulate, calc_ap, calc_tp
from nuscenes.eval.detection.constants import TP_METRICS
from nuscenes.eval.detection.render import summary_plot, class_pr_curve, class_tp_curve, dist_pr_curve, visualize_sample

   # Add center distances.
pred_boxes = add_center_dist(nusc, pred_boxes)
gt_boxes = add_center_dist(nusc, gt_boxes)

# -----------------------------------
# Step 1: Accumulate metric data for all classes and distance thresholds.
# -----------------------------------
if verbose:
    print('Accumulating metric data...')
metric_data_list = DetectionMetricDataList()

for class_name in cfg2.class_names:
    for dist_th in cfg2.dist_ths:
        continue
     




Accumulating metric data...


In [8]:
        
    
    
#Initialization

from nuscenes.eval.common.loaders import add_center_dist,filter_eval_boxes
from nuscenes.eval.tracking.loaders import create_tracks

# Add center distances.
pred_boxes = add_center_dist(nusc, pred_boxes)
gt_boxes = add_center_dist(nusc, gt_boxes)

print(pred_boxes)
print(gt_boxes)

# Filter boxes (distance, points per box, etc.).
if verbose:
            print('Filtering tracks')
            pred_boxes = filter_eval_boxes(nusc, pred_boxes, cfg.class_range, verbose=verbose)
if verbose:
            print('Filtering ground truth tracks')
            gt_boxes = filter_eval_boxes(nusc, gt_boxes, cfg.class_range, verbose=verbose)

            sample_tokens = gt_boxes.sample_tokens

            # Convert boxes to tracks format.
            tracks_gt = create_tracks(gt_boxes, nusc, eval_set, gt=True)
            tracks_pred = create_tracks(pred_boxes, nusc, eval_set, gt=False)


EvalBoxes with 708 boxes across 40 samples
EvalBoxes with 25 boxes across 40 samples
Filtering tracks
=> Original number of boxes: 708
=> After distance based filtering: 703
=> After LIDAR and RADAR points based filtering: 703
=> After bike rack filtering: 703
Filtering ground truth tracks
=> Original number of boxes: 25
=> After distance based filtering: 24
=> After LIDAR and RADAR points based filtering: 24
=> After bike rack filtering: 24


In [9]:
#Detection
class_name='car'
npos = len([1 for gt_boxes in gt_boxes.all if gt_boxes.tracking_name == 'car'])
if verbose:
        print("Found {} GT of class {} out of {} total across {} samples.".
              format(npos, class_name, len(gt_boxes.all), len(gt_boxes.sample_tokens)))
# Organize the predictions in a single list.
pred_boxes_list = [box for box in pred_boxes2.all if box.tracking_name == 'car']
pred_confs = [box.tracking_score for box in pred_boxes_list]
#print(pred_confs)
if verbose:
        print("Found {} PRED of class {} out of {} total across {} samples.".
              format(len(pred_confs), class_name, len(pred_boxes2.all), len(pred_boxes2.sample_tokens)))
                            
# Sort by confidence.
sortind = [i for (v, i) in sorted((v, i) for (i, v) in enumerate(pred_confs))][::-1]
sortind2 = [i for  (i, v) in enumerate(pred_confs)][::-1]
sortind3 = [i for (v, i) in ((v, i) for (i, v) in enumerate(pred_confs))]
print(sortind2)
# Do the actual matching.
tp = []  # Accumulator of true positives.
fp = []  # Accumulator of false positives.
conf = []  # Accumulator of confidences.


# match_data holds the extra metrics we calculate for each match.
match_data = {'trans_err': [],
                  'vel_err': [],
                  'scale_err': [],
                  'orient_err': [],
                  'attr_err': [],
                  'IOU': [],
                  'conf': []}

print('translation_gt = ',trans_alist)
print('size_gt = ',size_gt_alist)
print('rotation = ', rotation_gt_alist)



Found 24 GT of class car out of 24 total across 40 samples.
Found 708 PRED of class car out of 708 total across 40 samples.
[707, 706, 705, 704, 703, 702, 701, 700, 699, 698, 697, 696, 695, 694, 693, 692, 691, 690, 689, 688, 687, 686, 685, 684, 683, 682, 681, 680, 679, 678, 677, 676, 675, 674, 673, 672, 671, 670, 669, 668, 667, 666, 665, 664, 663, 662, 661, 660, 659, 658, 657, 656, 655, 654, 653, 652, 651, 650, 649, 648, 647, 646, 645, 644, 643, 642, 641, 640, 639, 638, 637, 636, 635, 634, 633, 632, 631, 630, 629, 628, 627, 626, 625, 624, 623, 622, 621, 620, 619, 618, 617, 616, 615, 614, 613, 612, 611, 610, 609, 608, 607, 606, 605, 604, 603, 602, 601, 600, 599, 598, 597, 596, 595, 594, 593, 592, 591, 590, 589, 588, 587, 586, 585, 584, 583, 582, 581, 580, 579, 578, 577, 576, 575, 574, 573, 572, 571, 570, 569, 568, 567, 566, 565, 564, 563, 562, 561, 560, 559, 558, 557, 556, 555, 554, 553, 552, 551, 550, 549, 548, 547, 546, 545, 544, 543, 542, 541, 540, 539, 538, 537, 536, 535, 534, 533, 

In [10]:
from typing import Callable
from nuscenes.eval.common.utils import center_distance, scale_iou, yaw_diff, velocity_l2, attr_acc, cummean
from nuscenes.eval.detection.data_classes import DetectionMetricDataList


    




# ---------------------------------------------
# Match and accumulate match data.
# ---------------------------------------------

taken = set()  # Initially no gt bounding box is matched.
for ind in sortind3:
    pred_box = pred_boxes_list[ind]
    min_dist = np.inf
    match_gt_idx = None
    for gt_idx, gt_box in enumerate(gt_boxes[pred_box.sample_token]):
        # Find closest match among ground truth boxes
        if gt_box.tracking_name == class_name and not (pred_box.sample_token, gt_idx) in taken:
            this_distance = cfg.dist_fcn_callable(gt_box, pred_box)
            if this_distance < min_dist:
                min_dist = this_distance
                match_gt_idx = gt_idx
                # add reference to closest annotation
                #sample_annotation_token = gt_boxes2.sample_annotation_token

    # If the closest match is close enough according to threshold we have a match!
    is_match = min_dist < dist_th

    if is_match:
        taken.add((pred_box.sample_token, match_gt_idx))
        #  Update tp, fp and confs.
        tp.append(1)
        fp.append(0)
        #conf.append(pred_box.tracking_score)
        # Since it is a match, update match data also.
        gt_box_match = gt_boxes2[pred_box.sample_token][match_gt_idx]
        match_data['IOU'].append(scale_iou(gt_box_match, pred_box))
        match_data['scale_err'].append(1 - scale_iou(gt_box_match, pred_box))
        match_data['trans_err'].append(center_distance(gt_box_match, pred_box))
        period = np.pi if class_name == 'barrier' else 2 * np.pi
        match_data['orient_err'].append(yaw_diff(gt_box_match, pred_box, period=period))
        match_data['conf'].append(pred_box.tracking_score)
        #print(yaw_diff(gt_box_match, pred_box, period=period))
    
#test
print('IOU= ',match_data['IOU'])
print('Orien= ',match_data['orient_err'])
print('trans= ',match_data['trans_err'])
print('conf= ',match_data['conf'])        
print('lidar_pts = ',lidar_pts_list)
print('visibility  =[ ',', '.join(map(str, visibility_alist)),']')
if verbose:
            print('Accumulating metric data...')
metric_data_list = DetectionMetricDataList()


             


IOU=  [0.7787860518810203, 0.871065854385429, 0.8189904684639117, 0.806385685411607, 0.811878290658615, 0.8531809900459301, 0.7885792138498778, 0.6668965539430053, 0.6604979126242967, 0.6869528073101462, 0.6869821954171611, 0.7575558566599698, 0.7500558195928099, 0.7695044887324232, 0.8322776261298592, 0.8039959000636802, 0.8189712305390463, 0.8426957172221892, 0.8348438868945531, 0.8371321163659546, 0.8188442868640353, 0.7891128580240994, 0.7891128580240994, 0.9125367433008625]
Orien=  [0.040698327119526034, 0.04011929275939119, 0.012214947386664399, 0.019085162003719347, 0.026270776419983655, 0.024000165702698695, 0.03364791052438676, 0.11713021092139986, 0.18565822048953873, 0.06121880862354967, 0.09721875137614866, 0.07575790192178555, 0.06646113423563182, 0.053936317323627136, 0.01798051209668916, 0.04639536339845751, 0.12684540010031764, 0.0406988336482077, 0.09093292704200717, 0.19406406672794985, 0.04863773182467046, 0.04198564130837079, 0.11761657556145844, 3.0557632957616025]

In [11]:


# ---------------------------------------------
# Calculate and interpolate precision and recall
# ---------------------------------------------
# Do the actual matching.
tp = []  # Accumulator of true positives.
fp = []  # Accumulator of false positives.
conf = []  # Accumulator of confidences.
# Accumulate.
tp = np.cumsum(tp).astype(float)
fp = np.cumsum(fp).astype(float)
conf = np.array(conf)

# Calculate precision and recall.
prec = tp / (fp + tp)
rec = tp / float(npos)


In [12]:
#To render instance

my_instance = nusc.get('instance', instance_to_find)
print(my_instance)
instance_token = my_instance['token']
#nusc.render_instance(instance_token)

#nusc.render_annotation(my_instance['first_annotation_token'])
#nusc.render_annotation(my_instance['last_annotation_token'])


{'token': '52504c169efd4a0bb1437bfb8ed5bdb9', 'category_token': 'fd69059b62a3469fbaef25340c0eab7f', 'nbr_annotations': 25, 'first_annotation_token': '708a713c6c54442ab317acd75132d110', 'last_annotation_token': '7deb4d5a415540a99d06605ac4c39a29'}


In [14]:
#To render class
import os
from typing import Any, List

import matplotlib.pyplot as plt
import numpy as np
from pandas import DataFrame
from pyquaternion import Quaternion

from nuscenes.eval.common.render import setup_axis
from nuscenes.utils.data_classes import Box


def render(self, events: DataFrame, timestamp: int, frame_gt: List[TrackingBox], frame_pred: List[TrackingBox]) \
            -> None:
        """
        Render function for a given scene timestamp
        :param events: motmetrics events for that particular
        :param timestamp: timestamp for the rendering
        :param frame_gt: list of ground truth boxes
        :param frame_pred: list of prediction boxes
        """
        # Init.
        print('Rendering {}'.format(timestamp))
        switches = events[events.Type == 'SWITCH']
        switch_ids = switches.HId.values
        fig, ax = plt.subplots()

        # Plot GT boxes.
        for b in frame_gt:
            color = 'k'
            box = Box(b.ego_translation, b.size, Quaternion(b.rotation), name=b.tracking_name, token=b.tracking_id)
            box.render(ax, view=np.eye(4), colors=(color, color, color), linewidth=1)

        # Plot predicted boxes.
        for b in frame_pred:
            box = Box(b.ego_translation, b.size, Quaternion(b.rotation), name=b.tracking_name, token=b.tracking_id)

            # Determine color for this tracking id.
            if b.tracking_id not in self.id2color.keys():
                self.id2color[b.tracking_id] = (float(hash(b.tracking_id + 'r') % 256) / 255,
                                                float(hash(b.tracking_id + 'g') % 256) / 255,
                                                float(hash(b.tracking_id + 'b') % 256) / 255)

            # Render box. Highlight identity switches in red.
            if b.tracking_id in switch_ids:
                color = self.id2color[b.tracking_id]
                box.render(ax, view=np.eye(4), colors=('r', 'r', color))
            else:
                color = self.id2color[b.tracking_id]
                box.render(ax, view=np.eye(4), colors=(color, color, color))

        # Plot ego pose.
        plt.scatter(0, 0, s=96, facecolors='none', edgecolors='k', marker='o')
        plt.xlim(-50, 50)
        plt.ylim(-50, 50)

        # Save to disk and close figure.
        fig.savefig(os.path.join(self.save_path, '{}.png'.format(timestamp)))
        plt.close(fig)




In [15]:
#To render GT for this instance with all prd


import time
from nuscenes.eval.tracking.data_classes import TrackingMetrics,TrackingMetricDataList,TrackingMetricData
from nuscenes.eval.tracking.algo import TrackingEvaluation

#Choose where to save results
output_dir = '/home/albhaisi/Desktop/Presentation/for_automatic_test/test1111'
start_time = time.time()
# -----------------------------------
# Step 1: Accumulate metric data for all classes and distance thresholds.
# -----------------------------------
if verbose:
            print('Render gt and prd for instance: ', instance_to_find, '....')
metric_data_list = TrackingMetricDataList()
def accumulate_class(curr_class_name):
            curr_ev = TrackingEvaluation(tracks_gt, tracks_pred, curr_class_name, cfg.dist_fcn_callable,
                                         cfg.dist_th_tp, cfg.min_recall,
                                         num_thresholds=TrackingMetricData.nelem,
                                         metric_worst=cfg.metric_worst,
                                         verbose=verbose,
                                         output_dir=output_dir,
                                         render_classes='car')
            curr_md = curr_ev.accumulate()
            metric_data_list.set(curr_class_name, curr_md)

for class_name in cfg.class_names:
            accumulate_class(class_name)

Render gt and prd for instance:  52504c169efd4a0bb1437bfb8ed5bdb9 ....
Computing metrics for class bicycle...

Computing metrics for class bus...

Computing metrics for class car...



  0%|          | 0/1 [00:00<?, ?it/s]

Rendering 1533201555449770
Rendering 1533201555899345
Rendering 1533201556399196
Rendering 1533201556898521
Rendering 1533201557398445
Rendering 1533201557898276
Rendering 1533201558398722
Rendering 1533201558899150
Rendering 1533201559398481
Rendering 1533201559948098
Rendering 1533201560398780
Rendering 1533201560899191
Rendering 1533201561400178
Rendering 1533201561950367
Rendering 1533201562400005
Rendering 1533201562899822
Rendering 1533201563399688
Rendering 1533201563949323
Rendering 1533201564449198
Rendering 1533201564949085
Rendering 1533201565448981
Rendering 1533201565948273
Rendering 1533201566448167
Rendering 1533201566949161
Rendering 1533201567449389
Rendering 1533201567948922
Rendering 1533201568448786
Rendering 1533201568948625
Rendering 1533201569449115
Rendering 1533201569949499
Rendering 1533201570449399
Rendering 1533201570898919
Rendering 1533201571399362
Rendering 1533201571949548
Rendering 1533201572449982
Rendering 1533201572949849
Rendering 1533201573449735
R

Computed thresholds



		MOTAR	MOTP	Recall	Frames	GT	GT-Mtch	GT-Miss	GT-IDS	Pred	Pred-TP	Pred-FP	Pred-IDS
thr_0.7440	0.000	0.326	0.958	40	24	23	1	0	103	23	80	0

Computing metrics for class motorcycle...

Computing metrics for class pedestrian...

Computing metrics for class trailer...

Computing metrics for class truck...



In [24]:
from nuscenes.eval.detection.data_classes import DetectionConfig, DetectionMetrics
import os

_label_aps = defaultdict(lambda: defaultdict(float))
metrics = DetectionMetrics(cfg2)

#Where to save results
output_dir = '/home/albhaisi/Desktop/Presentation/for_automatic_test/base_codes/test'

def mean_dist_aps():
        """ Calculates the mean over distance thresholds for each label. """
        return {class_name: np.mean(list(d.values())) for class_name, d in _label_aps.items()}

def mean_ap(self) -> float:
        """ Calculates the mean AP by averaging over distance thresholds and classes. """
        return float(np.mean(list(mean_dist_aps.values())))


if verbose:
    print('Saving metrics to: %s' % output_dir)

metrics_summary = metrics.serialize()
metrics_summary['meta'] = meta.copy()

with open(os.path.join(output_dir, 'metrics_details.json'), 'w') as f:
            f.write('Final_metrics_calculation: \n \n')

            f.write('Confidence_scor: \n \n')
            json.dump(match_data['conf'], f, indent=2)

            f.write('IOU: \n \n')
            json.dump(match_data['IOU'], f, indent=2)

            f.write('Translation_error: \n \n')
            json.dump(match_data['trans_err'], f, indent=2)
            
            f.write('Orientation_error: \n \n')
            json.dump(match_data['orient_err'], f, indent=2)

            f.write('Visibilty: \n \n')
            json.dump(', '.join(map(str, visibility_alist)), f, indent=2 )
            f.write('\n \n')
            f.write('Lidar points: \n \n')
            json.dump(', '.join(map(str, lidar_pts_list)), f, indent=2 )





Saving metrics to: /home/albhaisi/Desktop/Presentation/for_automatic_test/base_codes/test


/home/albhaisi/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/albhaisi/.local/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
